In [9]:
conda install -c conda-forge spacy

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.

# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/exceptions.py", line 1079, in __call__
        return func(*args, **kwargs)
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/cli/main.py", line 84, in _main
        exit_code = do_call(args, p)
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/cli/conda_argparse.py", line 83, in do_call
        return getattr(module, func_name)(args, parser)
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/cli/main_install.py", line 20, in execute
        install(args, parser, 'install')
      File "/opt/anaconda3/lib/python3.7/site-packages/conda/cli/install.py", line 116, in install
        if context.use_only_tar_bz2:
      File "/opt/anaconda3/lib/python3.7/site-pa

In [101]:
# New Stuff
from torchtext.data import Field, TabularDataset, BucketIterator, LabelField
import spacy
import torch

torch.backends.cudnn.deterministic = True

# spacy_en = spacy.load('en')

#     return [token.text for token in spacy_en.tokenizer(text)]

title = Field(sequential=True, use_vocab=True, tokenize='spacy', lower=True)
text = Field(sequential=True, use_vocab=True, tokenize='spacy', include_lengths = True, lower=True)
label = LabelField(dtype=torch.float)

fields = {'text': ('text', text), 'label': ('label', label), 'title': ('title', title)}

In [102]:
train_data = TabularDataset.splits(
    path='../data',
    train='news.csv',
    # validation='news.csv',
    # test='news.csv',
    format='csv',
    fields=fields)[0]
print("Num of training: ", len(train_data))
# print("Num of validation: ", len(validation_data))
# print("Num of testing: ", len(test_data))

Num of training:  6335


In [103]:
train_data, validation_data = train_data.split(split_ratio=0.65)
validation_data, test_data = validation_data.split(split_ratio=0.5)
print("Num of training: ", len(train_data))
print("Num of validation: ", len(validation_data))
print("Num of testing: ", len(test_data))

Num of training:  4118
Num of validation:  1108
Num of testing:  1109


In [104]:
vars(train_data[0])
# print(train_data[0])

{'text': ['hillary',
  'clinton',
  '’s',
  'presidential',
  'campaign',
  ',',
  'her',
  'network',
  'of',
  'super',
  'political',
  'action',
  'committees',
  ',',
  'and',
  'the',
  'liberal',
  'establishment',
  'relished',
  'a',
  'matchup',
  'against',
  'donald',
  'trump',
  '.',
  'however',
  ',',
  'her',
  'campaign',
  'failed',
  'to',
  'put',
  'forward',
  'an',
  'alternative',
  'for',
  'voters',
  'that',
  'would',
  'combat',
  'a',
  'candidate',
  'that',
  'tapped',
  'into',
  'the',
  'vast',
  'amount',
  'of',
  'disillusionment',
  'among',
  'citizens',
  '.',
  'tsunamis',
  'of',
  'voters',
  'unaccounted',
  'for',
  'in',
  'state',
  'polls',
  ',',
  'who',
  'do',
  'not',
  'identify',
  'with',
  'either',
  'the',
  'democratic',
  'or',
  'republican',
  'parties',
  ',',
  'made',
  'president',
  'trump',
  'a',
  'reality',
  '.',
  '\n',
  'clinton',
  '’s',
  'concession',
  'speech',
  'indicated',
  'the',
  'campaign',
  'an

In [122]:
MAX_VOCAB_SIZE = 25_000

text.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", 
unk_init = torch.Tensor.normal_)
title.build_vocab(train_data, max_size = MAX_VOCAB_SIZE, vectors = "glove.6B.100d", unk_init = torch.Tensor.normal_)
label.build_vocab(train_data)

In [123]:
BATCH_SIZE = 64

device = torch.device('cpu')

train_iterator, validation_iterator, test_iterator = BucketIterator.splits(
    (train_data, validation_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: x.title,
    sort_within_batch = True,
    # sort=False,
    device = device)

In [139]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
    bidirectional, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional=bidirectional, dropout = dropout )
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        #text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        # embedded = [sent len, batch size, emb dim]
        
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths, enforce_sorted=False)
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))

        #hidden = [batch size, hid dim * num directions]

        return self.fc(hidden)



In [140]:
INPUT_DIM = len(text.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = text.vocab.stoi[text.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)


In [141]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,810,857 trainable parameters


In [142]:
pretrained_embeddings = text.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [143]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 1.2854, -0.1562, -0.3448,  ...,  1.5273, -0.0496, -0.3139],
        [ 0.7317, -1.0654,  0.5485,  ...,  1.1625,  1.5158, -0.6169],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4451,  0.4952, -0.2546,  ...,  0.1101,  0.0458,  0.1567],
        [-0.2559, -0.2528,  0.0982,  ...,  0.3928, -0.0542,  0.0134],
        [ 0.4679,  0.0109,  1.0769,  ...,  0.3389, -0.1455,  0.8972]])

In [144]:
UNK_IDX = text.vocab.stoi[text.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.4451,  0.4952, -0.2546,  ...,  0.1101,  0.0458,  0.1567],
        [-0.2559, -0.2528,  0.0982,  ...,  0.3928, -0.0542,  0.0134],
        [ 0.4679,  0.0109,  1.0769,  ...,  0.3389, -0.1455,  0.8972]])


In [145]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [146]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [147]:
def binary_accuracy(preds, label):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == label).float()
    acc = correct.sum()/len(correct)
    return acc

In [148]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()

        text, text_lengths = batch.text
        # print("-------batch.text is: -------")
        # print(batch.text)
        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc =+ acc.item()
    
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [149]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            text, text_lengths = batch.text

            predictions = model(text, text.length).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [150]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elasped_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elasped_mins * 60))

    return elasped_mins, elapsed_secs

In [151]:
N_EPOCHS = 5

best_valid_loss = float('inf')

In [137]:
for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    validation_loss, validation_acc = evaluate(model, validation_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = validation_loss
        torch.save(mode.state_dict(), 'LSTM-Model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\Validation Loss: {validation:.3f} | Validation Acc: {validation_acc*100:.2f}%')

-------batch.text is: -------
(tensor([[ 189,  742,  652,  ...,  293,   75,   36],
        [1934,    8,  856,  ...,   18,  241, 2481],
        [   3, 4613, 2409,  ...,  161,    3,  426],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]]), tensor([2917,  584,   15,  905, 1447,  746, 1105,  987, 2171, 2362,  717, 2975,
        1836,  927,  704, 1919,  999, 2305,  861,  707,   89, 1061, 1172, 1220,
        1357,  916, 1011,  993, 1485,  998,  112,  488, 3538,  624,  475,  509,
         950, 1107,  649,  889, 3854,  742, 2812, 2433, 1209, 1077, 1216,  762,
        1918, 2387, 2510,  812,  109,  449, 1967,  924,  747, 1413,  772,  149,
        1434, 1347,  450,  716]))


RuntimeError: `lengths` array must be sorted in decreasing order when `enforce_sorted` is True. You can pass `enforce_sorted=False` to pack_padded_sequence and/or pack_sequence to sidestep this requirement if you do not need ONNX exportability.

In [138]:
model.load_state_dict(torch.load('LSTM-Model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

FileNotFoundError: [Errno 2] No such file or directory: 'LSTM-Model.pt'

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, text):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(text)]
    indexed = [text.vocab.stoi[t] for t in tokenized]
    